# Układy równań liniowych
# oraz różne sposoby ich rozwiązywania

## Przykładowy układ równań i sposób jego przedstawienia za pomocą macierzy
Taki układ równań:
$$
    \begin{cases}
        2.3x_1 - 3.4x_2 + 0.66x_3 = 13.1 \\ \\
        -0.23x_1 + 6.2x_2 + 12.2x_3 = 2.0 \\ \\
        22.12x_1 + 3.33x_2 - 7.89x_3 = -23.12
    \end{cases}
$$
można zapisać za pomocą macierzy w ten sposób:
$$
    \begin{gather}
        \begin{bmatrix}
            2.3 & -3.4 & 0.66 \\
            -0.23 & 6.2 & 12.2 \\
            22.12 & 3.33 & -7.89
        \end{bmatrix}
        *
        \begin{bmatrix}
            x_1 \\
            x_2 \\
            x_3
        \end{bmatrix}
        =
        \begin{bmatrix}
            13.1 \\
            2.0 \\
            -23.12
        \end{bmatrix}
    \end{gather}
$$
gdzie kolejno $ \bold{A} $ to macierz podstawowa (główna) układu, $ \bold{x} $ to wektor niewiadomych oraz $ \bold{b} $ to wektor wyrazów wolnych. Ogólnie równanie macierzowe przedstawiamy więc, w następujący sposób:

$$ \bold{A} * \bold{x} = \bold{b} $$

Spotkać się też można z macierzą zawierającą wyrazy wolne, należy to traktować jak wiele układów równań, gdzie każdy składa się z takich samych współczynników oraz dla każdego układu przypisujemy kolejne kolumny macierzy wyrazów wolnych.

## Analiza sposobów rozwiązywania układów macierzowych

### Dane testowe
Jako dane testowe przyjmujemy macierz pasmową. Macierz pasmowa (wstęgowa) to kwadratowa macierz rzadka, której wszystkie elementy są zerowe poza diagonalą i pasmem (wstęgą) wokół niej. Mając daną macierz $ {\displaystyle n\times n,}$ jej elementy $ {\displaystyle a_{i,j}} $ są niezerowe, gdy ${\displaystyle i-k_{1}\leqslant j\leqslant i+k_{2},}$ gdzie ${\displaystyle k_{1,2}\geqslant 0}$ określają tzw. szerokość pasma. Jednak aby uprościć obliczenia będziemy operować na macierzach w formacie pełnym.

Nasza macierz główna będzie wymiaru $108\times 108, k_1 = k_2 = 2$ oraz będzie zawierać następujące elementy:

$$
\[
  \bold{A} =
  \left[ {\begin{array}{cccc}
    14 & -1 & -1 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 14 & -1 & -1 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & -1 & 14 & -1 & -1 & 0 & 0 & \cdots & 0 \\
    0 & -1 & -1 & 14 & -1 & -1 & 0 & \cdots & 0 \\
    \vdots &  & \ddots & \ddots & \ddots & \ddots & \ddots &  & \vdots \\
    0 & \cdots & 0 & -1 & -1 & 14 & -1 & -1 & 0 \\
    0 & \cdots & 0 & 0 & -1 & -1 & 14 & -1 & -1 \\
    0 & \cdots & 0 & 0 & 0 & -1 & -1 & 14 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & -1 & -1 & 14 \\
  \end{array} } \right]
\]
$$

wektor wyrazów wolnych:

$$
\[
  \bold{b} =
  \left[ {\begin{array}{cccc}
    sin(5) \\
    sin(10) \\
    sin(15) \\
    \vdots \\
    sin(530) \\
    sin(535) \\
    sin(540)
  \end{array} } \right]
\]
$$


In [1]:
from math import sin
from matrix import Matrix


def generate_test_data(size: int, a_1: float, a_2: float, a_3: float, f_4: int):
    # print("N =", size, "\na1 =", a_1, "\na2 =", a_2, "\na3 =", a_3)

    a = Matrix(size)
    rows, cols = a.get_size()
    for i in range(rows):
        for j in range(cols):
            if i == j:
                a.set_at(i, j, a_1)

            else:
                distance = abs(i - j)
                if distance == 1:
                    a.set_at(i, j, a_2)

                if distance == 2:
                    a.set_at(i, j, a_3)

    b = Matrix(n, m=1)
    for i in range(b.get_size()[0]):
        b.set_at(i, value=sin(i * (f_4 + 1)))

    return a, b


my_index = "184934"
n = 9 * int(my_index[len(my_index) - 2]) * int(my_index[len(my_index) - 1])
a1 = 5 + float(my_index[3])
a2 = a3 = -1.0
f = int(my_index[2])
matrix_a, vector_b = generate_test_data(n, a1, a2, a3, f)

# print(matrix_a.head(1), vector_b.head(3))

### Metody iteracyjne
Jako pierwsze testować będziemy metody iteracyjne, a konkretnie metodę Jacobiego oraz metodą Gaussa-Seidla. Metody iteracyjne to metody, które nie uzyskują dokładnego rozwiązania, a tylko jego przybliżenie, w zamian za szybkość działania.

Bardzo ważne dla algorytmów iteracyjnych jest określenie, w jakim momencie osiągneliśmy satysfakcjonujące przybliżenie rozwiązania. W tym celu skorzystamy z tzw. *wektora residuum*, który dla *k*-tej iteracji przyjmuje postać:

$$
    \bold{res^{(k)}} = \bold{A}\bold{x}^{(k)} - \bold{b}
$$

Wielkość błędu natomiast jesteśmy w stanie zbadać określając jego normę euklidesową.

$$
    ||\bold{e}||_2 = \sqrt{\sum_{j=1}^{n}e_j^2}
$$

In [2]:
from math import sqrt


def euclidean_norm(vec: Matrix):
    return sqrt(sum([vec.get_at(i) ** 2 for i in range(vec.get_size()[0])]))


print(euclidean_norm(vector_b))

7.341106924346289


#### Metoda Jacobiego
Metoda Jacobiego to jedna z metod iteracyjnych. Do wyliczenia kolejnych wektorów przybliżeń, używany jest wektor z poprzedniej iteracji. Mimo, że część obecnego wektora jest już wyliczona.

Pierwszym krokiem jest rozbicie macierzy systemowej na trzy macierze: $ \bold{A} = -\bold{L} -\bold{U} + \bold{D} $, gdzie kolejno macierz $ \bold{L} $ to macierz trójkątna dolna, zawierająca liczby przeciwne **poniżej** głównej diagonali macierzy $ \bold{A} $, macierz $ \bold{U} $ to macierz trójkątna górna, zawierająca liczby przeciwne **powyżej** głównej diagonali macierzy $ \bold{A} $, macierz $ \bold{D} $ to macierz diagonalna, zawierająca liczby z głównej diagonali macierzy $ \bold{A} $.

Przekształcając równanie $ \bold{A}*\bold{x}=\bold{b} $, wykorzystując powyższą równość otrzymujemy:
$$
    (-\bold{L}-\bold{U}+\bold{D})\bold{x}=\bold{b}
$$
po serii przekształceń:
$$
    \bold{D}\bold{x} = (\bold{L}+\bold{U})\bold{x} + \bold{b} \\
    \bold{x} = \bold{D}^{-1}(\bold{L} + \bold{U})\bold{x} + \bold{D}^{-1}\bold{b} \\
$$
otrzymujemy ostateczny wzór:
$$
    \tilde{\bold{x}}^{(n+1)} = \bold{D}^{-1}(\bold{L} + \bold{U})\tilde{\bold{x}}^{(n)} + \bold{D}^{-1}\bold{b}
$$
gdzie $ (n) $ oznacza n-tą iteracje.

Metoda ta wykorzystuje szybkie obliczeniowo rozwiązywanie układów równań zapisanych za pomocą macierzy **diagonalnych**, opisane takim wzorem:
$$
    x_i = \frac{b_i}{a_{ii}}
$$
gdzie $ i $ to kolejne indeksy rzędów.

In [3]:
import time

from matrix import diagonal_solver


def jacobi(a: Matrix, b: Matrix, epsilon: float):
    r = Matrix(a.get_size()[0], m=1, value=1.0)
    it = 0

    u = -a.tri_u(1)
    l = -a.tri_l(1)
    d = a.diag()

    const_1 = diagonal_solver(d, l + u)
    const_2 = diagonal_solver(d, b)

    while True:
        r = const_1 * r + const_2

        residuum = a * r - b
        if euclidean_norm(residuum) <= epsilon:
            break

        it += 1
    return it, r

#### Metoda Gaussa-Seidla
Metoda Gaussa-Seidla to kolejna metoda iteracyjna, bardzo podobna to poprzednio opisanej.

In [4]:
from matrix import forward_substitution_solver


def gauss_seidel(a: Matrix, b: Matrix, epsilon: float):
    r = Matrix(a.get_size()[0], m=1, value=1.0)
    it = 0

    u = -a.tri_u(1)
    l = -a.tri_l(1)
    d = a.diag()

    const_1 = d - l
    const_2 = forward_substitution_solver(const_1, b)  # TODO comment

    while True:
        r = forward_substitution_solver(const_1, u * r) + const_2  # TODO comment

        residuum = a * r - b
        if euclidean_norm(residuum) <= epsilon:
            break

        it += 1
    return it, r

#### Porównanie metod


In [40]:
eps = 1e-9
start_time = time.time()
jacobi_iterations, _ = jacobi(matrix_a, vector_b, eps)
jacobi_time = (time.time() - start_time) * 1000
print("Jacobi:")
print("Time:", "%.2f" % jacobi_time + "ms")
print("Iterations:", jacobi_iterations, "\n")

start_time = time.time()
gauss_iterations, _ = gauss_seidel(matrix_a, vector_b, eps)
gauss_time = (time.time() - start_time) * 1000
print("Gauss-Seidel:")
print("Time:", "%.2f" % gauss_time + "ms")
print("Iterations:", gauss_iterations, "\n")

delta = abs(jacobi_time - gauss_time)
if gauss_time < jacobi_time:
    print("Gauss-Seidel method was faster by:", "%.2f" % delta + "ms")
else:
    print("Jacobi method was faster by:", "%.2f" % delta + "ms")

Jacobi:
Time: 226.53ms
Iterations: 20 

Gauss-Seidel:
Time: 248.71ms
Iterations: 14 

Jacobi method was faster by: 22.18ms


Czas działania obu metod jest bardzo podobny. Jeżeli uruchomimy obliczenia pare razy to przeważnie szybsza okaże się metoda Jacobiego, ale nie zawesze, natomiast ilość iteracji w jakiej otrzymaliśmy zadawalające przybliżenie jest mniejsza dla metody Gaussa-Seidla. Różnica ta najprawdopodobniej uwydatni się przy większych danych wejściowych.

Jednak metody iteracyjne nie są idealne. Otrzymaliśmy poprawne wyniki, ponieważ dobraliśmy dane testowe do metod. Zobaczmy co się stanie, gdy dobierzemy nieco inne dane testowe.

In [42]:
matrix_a, vector_b = generate_test_data(n, 3., -1., -1., f)

jacobi(matrix_a, vector_b, eps)
gauss_seidel(matrix_a, vector_b, eps)

OverflowError: (34, 'Numerical result out of range')

# TODO wyjaśnić co dlaczego i jak
Metoda ta pozwala otrzymać poprawne przybliżenia dla macierzy **diagonalnie dominujących**, czyli takich, dla których suma modułów wartości elementów w danym rzędzie, poza elementem na diagonali, jest mniejsza od modułu wartości na diagonali.
$$
    |a_{ii}| > \sum_{}^{}_{i\ne{j}}|a_{ij}|
$$
Jednym z warunków otrzymania prawidłowego przybliżenia jest tak samo macierz **diagnoalnie dominująca**.

### Metody bezpośrednie

## Źródła
    1. https://en.wikipedia.org/wiki/Band_matrix
    2. https://www.gaussianwaves.com/2013/05/solving-a-triangular-matrix-using-forward-backward-substitution/